Install the libraries

In [ ]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.9 MB/s eta 0:00:00


In [ ]:
from collections import defaultdict
import gdown
import gzip
import json
import random
import pickle

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import ticker
sns.set(style='ticks', font_scale=1.2)

In [ ]:
#import the libraries for importing and dealing with data
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import libraries for processing
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Load German Dataset and Clean it**

In [ ]:
#load and read the training TEXT From file
germantrainingSet = pd.read_csv('drive/MyDrive/germanGoogleTrans.txt', on_bad_lines='skip', sep=';', header=None)

In [ ]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
germantrainingSet=germantrainingSet.rename(columns={0: 'Tweet'})

In [ ]:
germantrainingSet.head()

,Tweet
0,"@corinnamilborn Dear Corinna, we would love to..."
1,@Martin28a You are also right. Our tweet was a...
2,@ahrens_theo happy greetings from the most bea...
3,@dushanwegner Americans would have voted for a...
4,@ spdde not a reliable negotiating partner. Na...


In [ ]:
def check_for_abuse(row):
    if row.endswith("ABUSE"):
        return 1
    else:
        return 0

In [ ]:
germantrainingSet['Label'] = germantrainingSet['Tweet'].apply(check_for_abuse)

In [ ]:
germantrainingSet['Tweet'] = germantrainingSet['Tweet'].apply(lambda x: ' '.join(x.split()[:-1]))

In [ ]:
germantrainingSet['Tweet'] = germantrainingSet['Tweet'].str.replace('OTHER', '')
germantrainingSet['Tweet'] = germantrainingSet['Tweet'].str.replace('OFFENSE', '')

In [ ]:
germantrainingSet.head()

,Tweet,Label
0,"@corinnamilborn Dear Corinna, we would love to...",0
1,@Martin28a You are also right. Our tweet was a...,0
2,@ahrens_theo happy greetings from the most bea...,0
3,@dushanwegner Americans would have voted for a...,0
4,@ spdde not a reliable negotiating partner. Na...,0


In [ ]:
germantrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4945 entries, 0 to 4944
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   4945 non-null   object
 1   Label   4945 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 77.4+ KB


In [ ]:
count = (germantrainingSet['Label'] == 1).sum()

# The condition (df['Label'] == 1) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 1 in 'Label' column:", count)

Number of rows with value 1 in 'Label' column: 1009


In [ ]:
countZero = (germantrainingSet['Label'] == 0).sum()

# The condition (df['Label'] == 0) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 0 in 'Label' column:", countZero)

Number of rows with value 0 in 'Label' column: 3936


In [ ]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

germantrainingSet["tweet_without_url"] = germantrainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [ ]:
germantrainingSet.head()

,Tweet,Label,tweet_without_url
0,"@corinnamilborn Dear Corinna, we would love to...",0,"@corinnamilborn Dear Corinna, we would love to..."
1,@Martin28a You are also right. Our tweet was a...,0,@Martin28a You are also right. Our tweet was a...
2,@ahrens_theo happy greetings from the most bea...,0,@ahrens_theo happy greetings from the most bea...
3,@dushanwegner Americans would have voted for a...,0,@dushanwegner Americans would have voted for a...
4,@ spdde not a reliable negotiating partner. Na...,0,@ spdde not a reliable negotiating partner. Na...


**Load Spanish Dataset and Clean it**

In [ ]:
#load and read the training TEXT From file
spanishtrainingSet = pd.read_csv('drive/MyDrive/spanishGoogleTrans.txt', on_bad_lines='skip', sep=';', header=None)

In [ ]:
spanishtrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5959 entries, 0 to 5958
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5959 non-null   object
 1   1       5959 non-null   object
 2   2       5959 non-null   object
 3   3       5959 non-null   object
 4   4       5959 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 232.9+ KB


In [ ]:
spanishtrainingSet.head

<bound method NDFrame.head of                           0   1  \
0     id=828025263321657348  ||   
1     id=828025128797741057  ||   
2     id=828025087815274496  ||   
3     id=828025006626058241  ||   
4     id=828024709761658880  ||   
...                     ...  ..   
5954  id=943971610670829570  ||   
5955  id=943971502784991232  ||   
5956  id=943971433847447552  ||   
5957  id=943971418001281025  ||   
5958  id=943971377190760454  ||   

                                                      2   3  4  
0     Ismael is egocentric because he goes crazy if ...  ||  0  
1     ..it took time for someone to pronounce a Cata...  ||  0  
2     (This is not a political and reasoned speech, ...  ||  0  
3     Very despised, yes, but all dressed in Spanish...  ||  1  
4     fag, explain to me why sometimes I don't under...  ||  1  
...                                                 ...  .. ..  
5954  Whether you are pro-independence or constituti...  ||  0  
5955  @Bernithedude Governmen

In [ ]:
#drop the columns not needed
spanishtrainingSet.drop(columns=0, inplace=True)
spanishtrainingSet.drop(columns=1, inplace=True)
spanishtrainingSet.drop(columns=3,inplace=True)

In [ ]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
spanishtrainingSet=spanishtrainingSet.rename(columns={2: 'Tweet'})
spanishtrainingSet=spanishtrainingSet.rename(columns={4: 'Label'})

In [ ]:
spanishtrainingSet

,Tweet,Label
0,Ismael is egocentric because he goes crazy if ...,0
1,..it took time for someone to pronounce a Cata...,0
2,"(This is not a political and reasoned speech, ...",0
3,"Very despised, yes, but all dressed in Spanish...",1
4,"fag, explain to me why sometimes I don't under...",1
...,...,...
5954,Whether you are pro-independence or constituti...,0
5955,@Bernithedude Government key to what if there ...,0
5956,@13Pardis @guenhwyvarblack Pugs are an ultra-f...,0
5957,Applause in Colau's party when he announces a ...,0


In [ ]:
spanishtrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5959 entries, 0 to 5958
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   5959 non-null   object
 1   Label   5959 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 93.2+ KB


In [ ]:
count = (spanishtrainingSet['Label'] == 1).sum()

# The condition (df['Label'] == 1) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 1 in 'Label' column:", count)

Number of rows with value 1 in 'Label' column: 1560


In [ ]:
countZero = (spanishtrainingSet['Label'] == 0).sum()

# The condition (df['Label'] == 0) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 0 in 'Label' column:", countZero)

Number of rows with value 0 in 'Label' column: 4399


In [ ]:
#convert the Label to int64 in order to match later
#spanishtrainingSet["Label"] = pd.to_numeric(spanishtrainingSet["Label"])

**Remove all links**

In [ ]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

spanishtrainingSet["tweet_without_url"] = spanishtrainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [ ]:
spanishtrainingSet.head()

,Tweet,Label,tweet_without_url
0,Ismael is egocentric because he goes crazy if ...,0,Ismael is egocentric because he goes crazy if ...
1,..it took time for someone to pronounce a Cata...,0,..it took time for someone to pronounce a Cata...
2,"(This is not a political and reasoned speech, ...",0,"(This is not a political and reasoned speech, ..."
3,"Very despised, yes, but all dressed in Spanish...",1,"Very despised, yes, but all dressed in Spanish..."
4,"fag, explain to me why sometimes I don't under...",1,"fag, explain to me why sometimes I don't under..."


**Load Italian Dataset and Clean it **

In [ ]:
#load and read the training TEXT From file
italiantrainingSet = pd.read_csv('drive/MyDrive/italianGoogleTrans.txt', on_bad_lines='skip', sep='\t')

In [ ]:
italiantrainingSet.head()

,id text hs stereotype
0,"2066 This too is terrorism, to put people in a..."
1,2045 @user @user in fact as long as they made ...
2,"611 Courier: Bribes, Mafia Forgotten capit..."
3,"1259 @user one by one, because when the Israel..."
4,949 Fun of the day? Finding Italian patriots ...


In [ ]:
italiantrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6839 entries, 0 to 6838
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id text hs stereotype  6839 non-null   object
dtypes: object(1)
memory usage: 53.6+ KB


In [ ]:
#drop the columns not needed
#italiantrainingSet.drop(columns='id', inplace=True)
#italiantrainingSet.drop(columns='stereotype', inplace=True)

In [ ]:
#rename the column text to 'Tweet' and column hs to 'Label
#italiantrainingSet=italiantrainingSet.rename(columns={'text ': 'Tweet'})
#italiantrainingSet=italiantrainingSet.rename(columns={'hs': 'Label'})
italiantrainingSet=italiantrainingSet.rename(columns={'id text hs stereotype': 'Tweet'})

In [ ]:
italiantrainingSet['Tweet'] = italiantrainingSet['Tweet'].apply(lambda x: ' '.join(x.split()[:-1]))

In [ ]:
italiantrainingSet

,Tweet
0,"2066 This too is terrorism, to put people in a..."
1,2045 @user @user in fact as long as they made ...
2,"611 Courier: Bribes, Mafia Forgotten capital B..."
3,"1259 @user one by one, because when the Israel..."
4,949 Fun of the day? Finding Italian patriots p...
...,...
6834,9340 The nation states must be ready to give u...
6835,9121 The interior minister of Germany #HorstSe...
6836,8549 #Salvini: In Italy too many have gone to ...
6837,9240 @user @user Anyone who rejoices in good f...


In [ ]:
def check_for_hate(row):
    if row.endswith("1"):
        return 1
    else:
        return 0

In [ ]:
italiantrainingSet['Label'] = italiantrainingSet['Tweet'].apply(check_for_hate)

In [ ]:
italiantrainingSet['Tweet'] = italiantrainingSet['Tweet'].apply(lambda x: ' '.join(x.split()[:-1]))

In [ ]:
italiantrainingSet

,Tweet,Label
0,"2066 This too is terrorism, to put people in a...",0
1,2045 @user @user in fact as long as they made ...,0
2,"611 Courier: Bribes, Mafia Forgotten capital B...",0
3,"1259 @user one by one, because when the Israel...",0
4,949 Fun of the day? Finding Italian patriots p...,0
...,...,...
6834,9340 The nation states must be ready to give u...,0
6835,9121 The interior minister of Germany #HorstSe...,0
6836,8549 #Salvini: In Italy too many have gone to ...,0
6837,9240 @user @user Anyone who rejoices in good f...,0


In [ ]:
italiantrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6839 entries, 0 to 6838
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   6839 non-null   object
 1   Label   6839 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 107.0+ KB


In [ ]:
count = (italiantrainingSet['Label'] == 1).sum()

# The condition (df['Label'] == 1) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 1 in 'Label' column:", count)

Number of rows with value 1 in 'Label' column: 2734


In [ ]:
countZero = (italiantrainingSet['Label'] == 0).sum()

# The condition (df['Label'] == 0) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 0 in 'Label' column:", countZero)

Number of rows with value 0 in 'Label' column: 4105


In [ ]:
italiantrainingSet

,Tweet,Label
0,"2066 This too is terrorism, to put people in a...",0
1,2045 @user @user in fact as long as they made ...,0
2,"611 Courier: Bribes, Mafia Forgotten capital B...",0
3,"1259 @user one by one, because when the Israel...",0
4,949 Fun of the day? Finding Italian patriots p...,0
...,...,...
6834,9340 The nation states must be ready to give u...,0
6835,9121 The interior minister of Germany #HorstSe...,0
6836,8549 #Salvini: In Italy too many have gone to ...,0
6837,9240 @user @user Anyone who rejoices in good f...,0


In [ ]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

italiantrainingSet["tweet_without_url"] = italiantrainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [ ]:
italiantrainingSet

,Tweet,Label,tweet_without_url
0,"2066 This too is terrorism, to put people in a...",0,"2066 This too is terrorism, to put people in a..."
1,2045 @user @user in fact as long as they made ...,0,2045 @user @user in fact as long as they made ...
2,"611 Courier: Bribes, Mafia Forgotten capital B...",0,"611 Courier: Bribes, Mafia Forgotten capital B..."
3,"1259 @user one by one, because when the Israel...",0,"1259 @user one by one, because when the Israel..."
4,949 Fun of the day? Finding Italian patriots p...,0,949 Fun of the day? Finding Italian patriots p...
...,...,...,...
6834,9340 The nation states must be ready to give u...,0,9340 The nation states must be ready to give u...
6835,9121 The interior minister of Germany #HorstSe...,0,9121 The interior minister of Germany #HorstSe...
6836,8549 #Salvini: In Italy too many have gone to ...,0,8549 #Salvini: In Italy too many have gone to ...
6837,9240 @user @user Anyone who rejoices in good f...,0,9240 @user @user Anyone who rejoices in good f...


#Delete the first four characters in each cell of the column tweet_without_url to get rid of the ids

In [ ]:
italiantrainingSet["tweet_without_url"] = italiantrainingSet["tweet_without_url"].str[4:]

In [ ]:
italiantrainingSet

,Tweet,Label,tweet_without_url
0,"2066 This too is terrorism, to put people in a...",0,"This too is terrorism, to put people in a sta..."
1,2045 @user @user in fact as long as they made ...,0,@user @user in fact as long as they made mone...
2,"611 Courier: Bribes, Mafia Forgotten capital B...",0,"Courier: Bribes, Mafia Forgotten capital Bribe..."
3,"1259 @user one by one, because when the Israel...",0,"@user one by one, because when the Israelite ..."
4,949 Fun of the day? Finding Italian patriots p...,0,Fun of the day? Finding Italian patriots prais...
...,...,...,...
6834,9340 The nation states must be ready to give u...,0,The nation states must be ready to give up th...
6835,9121 The interior minister of Germany #HorstSe...,0,The interior minister of Germany #HorstSeehof...
6836,8549 #Salvini: In Italy too many have gone to ...,0,#Salvini: In Italy too many have gone to thei...
6837,9240 @user @user Anyone who rejoices in good f...,0,@user @user Anyone who rejoices in good faith...


In [ ]:
italiantrainingSet.head()

,Tweet,Label,tweet_without_url
0,"2066 This too is terrorism, to put people in a...",0,"This too is terrorism, to put people in a sta..."
1,2045 @user @user in fact as long as they made ...,0,@user @user in fact as long as they made mone...
2,"611 Courier: Bribes, Mafia Forgotten capital B...",0,"Courier: Bribes, Mafia Forgotten capital Bribe..."
3,"1259 @user one by one, because when the Israel...",0,"@user one by one, because when the Israelite ..."
4,949 Fun of the day? Finding Italian patriots p...,0,Fun of the day? Finding Italian patriots prais...


**Merge the Data**

In [ ]:
germantrainingSet.shape

(4945, 3)

In [ ]:
#drop the columns not needed
germantrainingSet.drop(columns='Tweet', inplace=True)
#germantrainingSet.drop(columns='tweet_w/o_url', inplace=True)
#germantrainingSet.drop(columns='tweet_lem', inplace=True)

In [ ]:
germantrainingSet.head()

,Label,tweet_without_url
0,0,"@corinnamilborn Dear Corinna, we would love to..."
1,0,@Martin28a You are also right. Our tweet was a...
2,0,@ahrens_theo happy greetings from the most bea...
3,0,@dushanwegner Americans would have voted for a...
4,0,@ spdde not a reliable negotiating partner. Na...


In [ ]:
#figure out the shape of spanish set
spanishtrainingSet.shape

(5959, 3)

In [ ]:
#drop the columns not needed
spanishtrainingSet.drop(columns='Tweet', inplace=True)
#spanishtrainingSet.drop(columns='tweet_w/o_url', inplace=True)
#spanishtrainingSet.drop(columns='tweet_lem', inplace=True)

In [ ]:
spanishtrainingSet.head()

,Label,tweet_without_url
0,0,Ismael is egocentric because he goes crazy if ...
1,0,..it took time for someone to pronounce a Cata...
2,0,"(This is not a political and reasoned speech, ..."
3,1,"Very despised, yes, but all dressed in Spanish..."
4,1,"fag, explain to me why sometimes I don't under..."


In [ ]:
#figure out the shape of italian set
italiantrainingSet.shape

(6839, 3)

In [ ]:
#drop the columns not needed
italiantrainingSet.drop(columns='Tweet', inplace=True)
#italiantrainingSet.drop(columns='tweet_w/o_url', inplace=True)
#italiantrainingSet.drop(columns='tweet_lem', inplace=True)

In [ ]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
italiantrainingSet=italiantrainingSet.rename(columns={1: 'tweet_without_url'})


In [ ]:
italiantrainingSet.head()

,Label,tweet_without_url
0,0,"This too is terrorism, to put people in a sta..."
1,0,@user @user in fact as long as they made mone...
2,0,"Courier: Bribes, Mafia Forgotten capital Bribe..."
3,0,"@user one by one, because when the Israelite ..."
4,0,Fun of the day? Finding Italian patriots prais...


In [ ]:
germanspanish_merged = pd.merge(germantrainingSet, spanishtrainingSet, how="outer", on=["Label", "tweet_without_url"])

In [ ]:
germanspanish_merged.shape

(10904, 2)

In [ ]:
germanspanish_merged.head()

,Label,tweet_without_url
0,0,"@corinnamilborn Dear Corinna, we would love to..."
1,0,@Martin28a You are also right. Our tweet was a...
2,0,@ahrens_theo happy greetings from the most bea...
3,0,@dushanwegner Americans would have voted for a...
4,0,@ spdde not a reliable negotiating partner. Na...


In [ ]:
print(germanspanish_merged)

       Label                                  tweet_without_url
0          0  @corinnamilborn Dear Corinna, we would love to...
1          0  @Martin28a You are also right. Our tweet was a...
2          0  @ahrens_theo happy greetings from the most bea...
3          0  @dushanwegner Americans would have voted for a...
4          0  @ spdde not a reliable negotiating partner. Na...
...      ...                                                ...
10899      0  Whether you are pro-independence or constituti...
10900      0  @Bernithedude Government key to what if there ...
10901      0  @13Pardis @guenhwyvarblack Pugs are an ultra-f...
10902      0  Applause in Colau's party when he announces a ...
10903      0  @Rafa_Hernando The Constitution is pro-indepen...

[10904 rows x 2 columns]


In [ ]:
germanspanishitalian_merged = pd.merge(germanspanish_merged, italiantrainingSet, how="outer", on=["Label", "tweet_without_url"])

In [ ]:
germanspanishitalian_merged.shape

(17743, 2)

In [ ]:
print(germanspanishitalian_merged)

       Label                                  tweet_without_url
0          0  @corinnamilborn Dear Corinna, we would love to...
1          0  @Martin28a You are also right. Our tweet was a...
2          0  @ahrens_theo happy greetings from the most bea...
3          0  @dushanwegner Americans would have voted for a...
4          0  @ spdde not a reliable negotiating partner. Na...
...      ...                                                ...
17738      0   The nation states must be ready to give up th...
17739      0   The interior minister of Germany #HorstSeehof...
17740      0   #Salvini: In Italy too many have gone to thei...
17741      0   @user @user Anyone who rejoices in good faith...
17742      0   Christian youth in #Ethiopia are lured by Isl...

[17743 rows x 2 columns]


In [ ]:
germanspanishitalian_merged.head()

,Label,tweet_without_url
0,0,"@corinnamilborn Dear Corinna, we would love to..."
1,0,@Martin28a You are also right. Our tweet was a...
2,0,@ahrens_theo happy greetings from the most bea...
3,0,@dushanwegner Americans would have voted for a...
4,0,@ spdde not a reliable negotiating partner. Na...


Rename Label to 'label'

In [ ]:
germanspanishitalian_merged = germanspanishitalian_merged.rename(columns={'Label': 'label'})

In [ ]:
germanspanishitalian_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17743 entries, 0 to 17742
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   label              17743 non-null  int64 
 1   tweet_without_url  17743 non-null  object
dtypes: int64(1), object(1)
memory usage: 415.9+ KB


In [ ]:
count = (germanspanishitalian_merged['label'] == 1).sum()

# The condition (df['Label'] == 1) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 1 in 'Label' column:", count)

Number of rows with value 1 in 'Label' column: 5303


In [ ]:
countZero = (germanspanishitalian_merged['label'] == 0).sum()

# The condition (df['Label'] == 0) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 0 in 'Label' column:", countZero)

Number of rows with value 0 in 'Label' column: 12440


In [ ]:
#germanspanishitalian_merged

In [ ]:
X = germanspanishitalian_merged["tweet_without_url"].values.tolist()

In [ ]:
y = germanspanishitalian_merged['label'].values.tolist()

**Split the text**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 0)

In [ ]:
train_texts = X_train
train_labels = y_train

test_texts = X_test
test_labels = y_test

In [ ]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(14194, 14194, 3549, 3549)

In [ ]:
train_labels[0], train_texts[0]

(0,
 ' @user And the Roma settlement between Togliatti and via dei Romanisti? Seeing people defecate when I look out the window is humiliating!')

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [ ]:
#import svm model and metrics
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
#model = LogisticRegression(max_iter=1000).fit(X_train, train_labels)
#SVM BASE LINE
#model = SVC(kernel='linear', C=100, gamma='auto').fit(X_train, train_labels)
model = SVC(kernel='linear', gamma='auto').fit(X_train, train_labels)
predictions = model.predict(X_test)

In [ ]:
print(classification_report(test_labels, predictions))

              precision    recall  f1-score   support

           0       0.80      0.91      0.86      2467
           1       0.71      0.49      0.58      1082

    accuracy                           0.79      3549
   macro avg       0.76      0.70      0.72      3549
weighted avg       0.78      0.79      0.77      3549



In [ ]:
#calculate F1 score
from sklearn.metrics import f1_score
f1_score(test_labels, predictions)

0.5843801201529218

In [ ]:
from sklearn.metrics import matthews_corrcoef
#calculate Matthews correlation coefficient
matthews_corrcoef(test_labels, predictions)

0.4599697830305465

In [ ]:
# This is the name of the BERT model that we want to use.
model_name = 'distilbert-base-cased'

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'

# This is the maximum number of tokens in any document sent to BERT.
max_length = 512

# This is the name of the directory where we'll save our model. You can name it whatever you want.
cached_model_directory_name = 'MasterDistilBERTGoogleTrans-model-saved'

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
#tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

In [ ]:
train_labels


[0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,


In [ ]:
unique_labels = set(label for label in train_labels)
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

In [ ]:
label2id.keys()

dict_keys([0, 1])

In [ ]:
id2label.keys()

dict_keys([0, 1])

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)

train_labels_encoded = [label2id[y] for y in train_labels]
test_labels_encoded  = [label2id[y] for y in test_labels]

In [ ]:
set(train_labels_encoded)

{0, 1}

In [ ]:
set(test_labels_encoded)

{0, 1}

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [ ]:
# The model_name needs to match the name used for the tokenizer above.
#model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier

In [ ]:
#!pip install --upgrade accelerate

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.633500,0.585273,0.696816
200,0.576200,0.514076,0.741617
300,0.488300,0.488740,0.761905
400,0.483600,0.477584,0.770358
500,0.458300,0.485605,0.768385
600,0.501900,0.484718,0.757678
700,0.473900,0.496266,0.759651
800,0.481600,0.459766,0.785010
900,0.479900,0.452391,0.786137
1000,0.469900,0.457499,0.790082


TrainOutput(global_step=3550, training_loss=0.417699372734822, metrics={'train_runtime': 618.5909, 'train_samples_per_second': 45.891, 'train_steps_per_second': 5.739, 'total_flos': 2005102093870512.0, 'train_loss': 0.417699372734822, 'epoch': 2.0})

In [ ]:
trainer.save_model(cached_model_directory_name)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.4913136661052704,
 'eval_accuracy': 0.7999436460974922,
 'eval_runtime': 9.3987,
 'eval_samples_per_second': 377.605,
 'eval_steps_per_second': 18.939,
 'epoch': 2.0}

In [ ]:
predicted_results = trainer.predict(test_dataset)

In [ ]:
predicted_results.predictions.shape

(3549, 2)

In [ ]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [ ]:
len(predicted_labels)

3549

Print the Results for GoogleTrans + DistilBERT

In [ ]:
print(classification_report(test_labels,
                            predicted_labels))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      2467
           1       0.69      0.63      0.66      1082

    accuracy                           0.80      3549
   macro avg       0.77      0.75      0.76      3549
weighted avg       0.80      0.80      0.80      3549



In [ ]:
#calculate F1 score
from sklearn.metrics import f1_score
f1_score(test_labels, predicted_labels)

0.6563407550822846

In [ ]:
from sklearn.metrics import matthews_corrcoef
#calculate Matthews correlation coefficient
matthews_corrcoef(test_labels, predicted_labels)

0.5168472916930199